## Exercise 09 — Plotly

Animated commit dynamics for `project1` ready submissions using Plotly (adapted from the [line race example](https://github.com/datageekrj/YouTubeChannelHostingFiles/blob/master/lineRace.py)).

In [7]:
import pandas as pd
import sqlite3
import plotly.graph_objects as go
import numpy as np

DB_PATH = '../data/checking-logs.sqlite'
PLOT_BG = '#d9d9d9'
FIG_HEIGHT = 600
FIG_WIDTH = int(FIG_HEIGHT * 1.5)

In [8]:
with sqlite3.connect(DB_PATH) as connection:
    checker_info = pd.read_sql_query("PRAGMA table_info('checker');", connection)
checker_info

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,status,TEXT,0,None,0
2,2,success,INTEGER,0,None,0
3,3,timestamp,TIMESTAMP,0,None,0
4,4,numTrials,INTEGER,0,None,0
5,5,labname,TEXT,0,None,0
6,6,uid,TEXT,0,None,0


In [9]:
with sqlite3.connect(DB_PATH) as connection:
    project_commits = pd.read_sql_query(
        """
        SELECT uid, timestamp
        FROM checker
        WHERE uid LIKE 'user_%'
          AND labname = 'project1'
          AND status = 'ready'
        ORDER BY timestamp
        """,
        connection,
        parse_dates=['timestamp']
    )
project_commits.head()

,uid,timestamp
0,user_4,2020-04-17 05:19:02.744528
1,user_4,2020-04-17 05:22:45.549397
2,user_4,2020-04-17 05:34:24.422370
3,user_4,2020-04-17 05:43:27.773992
4,user_4,2020-04-17 05:46:32.275104


In [10]:
project_commits['timestamp'] = pd.to_datetime(project_commits['timestamp'])
project_commits = project_commits.sort_values('timestamp').reset_index(drop=True)
project_commits['num_trials'] = project_commits.groupby('uid').cumcount() + 1
project_commits.head()

,uid,timestamp,num_trials
0,user_4,2020-04-17 05:19:02.744528,1
1,user_4,2020-04-17 05:22:45.549397,2
2,user_4,2020-04-17 05:34:24.422370,3
3,user_4,2020-04-17 05:43:27.773992,4
4,user_4,2020-04-17 05:46:32.275104,5


In [11]:

# Aggregate commits by day to replicate the reference race plot
project_commits['date'] = project_commits['timestamp'].dt.normalize()
daily_timeline = (
    project_commits.groupby(['date', 'uid']).size()
    .unstack(fill_value=0)
    .sort_index()
    .cumsum()
)
user_order = (
    project_commits.groupby('uid')['num_trials'].max()
    .sort_values(ascending=False)
    .index.tolist()
)
if daily_timeline.empty:
    timeline = daily_timeline
    step_numbers = np.array([])
    date_labels = []
else:
    timeline = daily_timeline.reindex(columns=user_order).fillna(0)
    step_numbers = np.arange(1, len(timeline) + 1)
    date_labels = [idx.strftime('%Y-%m-%d') for idx in timeline.index]
len(date_labels), timeline.head() if not timeline.empty else timeline


(19,
 uid         user_4  user_14  user_2  user_25  user_26  user_10  user_3  \
 date                                                                     
 2020-04-17       7        0       0        0        0        0       0   
 2020-04-18       7        0       0        0        0        0       0   
 2020-04-19      11        0       0        0        0        0       0   
 2020-04-22      11        0       0        0        0        0       0   
 2020-04-23      20        0       0        0        0        0       0   
 
 uid         user_20  user_29  user_24  ...  user_21  user_27  user_8  user_31  \
 date                                   ...                                      
 2020-04-17        0        0        0  ...        0        0       0        0   
 2020-04-18        0        0        0  ...        0        0       0        0   
 2020-04-19        0        0        0  ...        0        0       0        0   
 2020-04-22        0        0        0  ...        0      

In [12]:

# Prepare animation components that closely match the provided reference
if timeline.empty:
    fig = go.Figure()
else:
    x_values = step_numbers
    initial_traces = [
        go.Scatter(
            x=x_values[:1],
            y=timeline[user].iloc[:1],
            mode='lines+markers',
            name=user,
            line=dict(width=3)
        )
        for user in user_order
    ]

    frames = []
    for idx in range(len(x_values)):
        frame_x = x_values[: idx + 1]
        frame_data = []
        for user in user_order:
            frame_data.append(
                go.Scatter(
                    x=frame_x,
                    y=timeline[user].iloc[: idx + 1],
                    mode='lines+markers',
                    line=dict(width=3),
                    name=user
                )
            )
        frames.append(go.Frame(data=frame_data, name=str(idx)))

    play_button = dict(
        label='play',
        method='animate',
        args=[None, {
            'frame': {'duration': 300, 'redraw': True},
            'transition': {'duration': 0},
            'fromcurrent': True
        }]
    )
    pause_button = dict(
        label='pause',
        method='animate',
        args=[[None], {
            'frame': {'duration': 0, 'redraw': False},
            'mode': 'immediate',
            'transition': {'duration': 0}
        }]
    )

    slider_steps = []
    for idx, label in enumerate(date_labels):
        slider_steps.append(dict(
            method='animate',
            args=[[str(idx)], {
                'frame': {'duration': 0, 'redraw': True},
                'mode': 'immediate',
                'transition': {'duration': 0}
            }],
            label=label
        ))

    max_val = float(timeline.values.max()) if len(timeline.values) else 1.0
    fig = go.Figure(
        data=initial_traces,
        frames=frames,
        layout=go.Layout(
            title=dict(
                text='<i>Dynamic of commits per user in project1</i>',
                font=dict(size=26)
            ),
            xaxis=dict(title='', range=[0, max(x_values) + 1], tickmode='linear'),
            yaxis=dict(title='', range=[0, max_val * 1.05]),
            height=FIG_HEIGHT,
            width=FIG_WIDTH,
            plot_bgcolor=PLOT_BG,
            paper_bgcolor=PLOT_BG,
            updatemenus=[dict(
                type='buttons',
                showactive=False,
                buttons=[play_button, pause_button],
                x=0.0,
                y=1.1,
                xanchor='left',
                yanchor='top'
            )],
            sliders=[dict(
                active=0,
                steps=slider_steps,
                x=0.1,
                y=-0.05,
                len=0.8
            )],
            legend=dict(title='uid', orientation='v', x=1.02, y=1.0)
        )
    )
fig
